# 앙상블 학습
- 정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘

## 랜덤 포레스트
- 앙상블 학습의 대표적인 모델
- 여러 개의 결정트리(Decision Tree)를 조합하여 예측 성능을 높이는 앙상블(Ensemble) 학습 기법이다.
- 원본 데이터에서 중복을 허용하면서 랜덤 샘플링을 하는 부트스트랩 샘플링을 진행한다.
- RandomForestClassifier는 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택한다. 즉 4개의 특성이 있다면 노드마다 2개를 랜덤하게 선택해 사용
  - RandomForestRegressor는 전체 특성 사용
- 분류일때는 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측한다.
  - 회귀일 땐 단순히 각 트리의 예측을 평균한다.
- 랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용하므로 과대적합을 막아주고, 검증과 테스트 세트에서 안정적인 성능을 얻을 수 있다.

In [75]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv("https://bit.ly/wine_csv_data")
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine[['class']].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(
    data, target, random_state=42)

In [76]:
# 교차 검증 수행. 여기선 cross_validate()함수 사용
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
# return_train_score=True로 지정하면 검증 점수뿐만 아니라 훈련세트에 대한 점수도 같이 반환한다.

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.997844759088341 0.8914208392565683


- 랜덤 포레스트는 결정 트리의 앙상블이므로 DecisionTreeClassifier가 제공하는 중요한 매개 변수를 모두 제공한다.
- 결정 트리의 큰 장점 중 하나인 특성 중요도를 계산한다.
  - 특성 중요도: 각 결정 트리의 특성 중요도를 취합한 것

In [78]:
# 특성 중요도 출력
rf.fit(train_input, train_target)
print(rf.feature_importances_)

C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[0.23155241 0.49706658 0.27138101]


- 앞의 결정트리와 다른 결과가 나왔는데, 이는 랜덤포레스트가 특성의 일부를 랜덤하게 선택해 결정 트리를 훈련하기 때문이다.
- 이는 과대적합을 줄이고 일반화 성능을 높이는데 도움이 된다.

### 자체 검증
- 랜덤 포레스트는 OOB(남는 샘플 데이터)를 통해 부트스트랩 샘플로 훈련한 결정 트리를 평가할 수 있다.
  - RandomForestClassifier의 oob_score=True로 지정해줘야한다.

In [80]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.8981937602627258


## 엑스트라 트리
- 랜덤 포레스트와 다르게 부트스트랩 샘플을 사용하지 않는다. 즉 전체 훈련 세트를 사용한다.
- 대신 노드를 분할할 때 가장 좋은 분할을 찾는것이 아닌 무작위 분할을 한다.
- 많은 트리를 학습하므로 과대적합을 방지하고 검증 세트의 점수를 높일 수 있다.
- 보통 엑스트라 트리가 랜덤 포레스트보다 무작위성이 커 더 많은 결정트리를 훈련해야한다.
- 랜덤하게 노드를 분할하기 때문에 빠른 계산 속도가 장점이다.

In [82]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.997844759088341 0.8903937240035804


In [84]:
# 특성 중요도 파악
et.fit(train_input, train_target)
print(et.feature_importances_)

C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[0.20702369 0.51313261 0.2798437 ]


## 그레디언트 부스팅
- 깊이가 얕은 결정 트리를 사용해 이전 트리의 오차를 보완하는 방식
- GradientBoostingClassifier는 기본적으로 깊이가 3인 결정 트리 100개를 사용한다.
- 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있다.
- 경사 하강법을 사용해 트리 앙상블에 추가한다.

In [86]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8894704231708938 0.8715107671247301


In [87]:
# 결정 트리의 개수를 늘려도 과대적합에 매우 강하다.
# 학습률을 증가시키고 트리의 개수를 늘려보자(n_estimators=500, learning_rate=0.2)
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9512006117505237 0.879719686200179


In [88]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

# 마찬가지로 당도에 좀 더 집중하는 모습을 볼 수 있다.

C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0.1479308  0.68876707 0.16330213]


## 히스토그램 기반 그레이디언트 부스팅
- 정형 데이터를 다루는 머신러닝 알고리즘 중 가장 많이 쓰인다.
- 입력 특성을 256개의 구간으로 나눔 -> 노드 분할 시 최적의 분할을 매우 빠르게 찾음
- 256개 중 하나를 떼어 놓고 결측치값을 위해 사용함 -> 전처리 과정 축소
- HistGradientBoostingClassifier 사용

In [90]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

0.9380129799494501 0.8805410414363187


- 특성 중요도를 계산하기 위해 permutation_importance() 함수 사용
- n_repeats 매개변수는 랜덤하게 섞을 횟수 지정

In [93]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)

print(result.importances_mean)

C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0.09144089 0.23493432 0.08760263]


- 조금 더 당도에 집중하고있음을 알 수 있다.

In [110]:
# 테스트 세트에서에도 특성 중요도 파악
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)

print(result.importances_mean)

[0.04141538 0.19827692 0.03870769]


In [111]:
# 테스트 세트에서의 성능 확인
hgb.score(test_input, test_target)

0.8584615384615385

### XGBoost
- 그레이디언트 부스팅 알고리즘을 구현한 라이브러리 중 하나
- tree_method 매개변수를 hist로 지정해 히스토그램 기반 그레이디언트 부스팅 사용 가능

### LightGBM
- 마이크로스프트에서 만든 그레이디언트 부스팅 알고리즘을 구현한 라이브러리 중 하나